<a href="https://colab.research.google.com/github/Rashmi-debug43/Statathon/blob/Semantic-Search/Semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================
# 1. IMPORT LIBRARIES
# ======================================
import pandas as pd
import numpy as np
import gradio as gr
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


# ======================================
# 2. LOAD DATASET
# ======================================
df = pd.read_csv("/content/NSS.csv.csv")
df.columns = df.columns.str.strip()


# ======================================
# 3. LOAD EMBEDDING MODEL
# ======================================
model = SentenceTransformer("all-MiniLM-L6-v2")


# ======================================
# 4. CREATE EMBEDDINGS
# ======================================
descriptions = df["Description"].astype(str).tolist()
description_embeddings = model.encode(
    descriptions,
    show_progress_bar=True,
    convert_to_numpy=True
)


# ======================================
# 5. CONFIDENCE LEVEL FUNCTION
# ======================================
def confidence_label(score):
    if score >= 0.75:
        return "High", "🟢"
    elif score >= 0.55:
        return "Medium", "🟡"
    else:
        return "Low", "🔴"


# ======================================
# 6. TOP-K MATCH FUNCTION
# ======================================
def identify_top_k_occupations(user_input, k):
    if user_input.strip() == "":
        return "⚠️ Please enter a job description."

    query_embedding = model.encode([user_input], convert_to_numpy=True)
    similarities = cosine_similarity(query_embedding, description_embeddings)[0]

    top_k_indices = similarities.argsort()[-k:][::-1]

    output_text = ""

    for idx in top_k_indices:
        score = similarities[idx]
        level, emoji = confidence_label(score)
        row = df.iloc[idx]

        output_text += f"""
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🎯 Confidence Score : {score:.4f}
📊 Confidence Level : {emoji} {level}
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔖 KEY CLASSIFICATION
Occupational Title : {row['Title']}
NCO 2015           : {row['Code']}

📄 FULL RECORD
S No               : {idx + 1}
Occupational Title : {row['Title']}
NCO 2015           : {row['Code']}
Description        : {row['Description']}
"""

    return output_text


# ======================================
# 7. CLEAR FUNCTION
# ======================================
def clear_all():
    return "", 5, ""


# ======================================
# 8. GRADIO UI (TEXT ONLY)
# ======================================
with gr.Blocks() as demo:

    gr.HTML("""
    <style>
        .gradio-container {
            max-width: 1200px !important;
        }
    </style>
    """)

    gr.Markdown("""
    # 🔍 Top-K Semantic Occupational Mapping (MOSPI)
    ### AI-based Job → NCO 2015 Mapping
    *Sentence-BERT • Confidence Levels*
    """)

    user_input = gr.Textbox(
        lines=5,
        label="📝 Job Description",
        placeholder="Eg: auditor, accounts executive, billing clerk..."
    )

    top_k = gr.Slider(1, 10, value=5, step=1, label="🔢 Top-K Matches")

    with gr.Row():
        submit_btn = gr.Button("🚀 Submit", variant="primary")
        clear_btn = gr.Button("🧹 Clear")

    output_text = gr.Textbox(
        label="📄 Occupation Classification (Detailed)",
        lines=25
    )

    submit_btn.click(
        identify_top_k_occupations,
        inputs=[user_input, top_k],
        outputs=output_text
    )

    clear_btn.click(
        clear_all,
        outputs=[user_input, top_k, output_text]
    )


# ======================================
# 9. LAUNCH
# ======================================
demo.launch(debug=True)
